In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import talib as ta 
from talib import RSI
from talib import BBANDS
import matplotlib.pyplot as plt
from matplotlib import style
import requests
style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
import yfinance as yf

In [ ]:
a1 = yf.Ticker("AAPl")
df1 = df1 = a1.history(period="3000d" , interval = '1d')
dfa = a1.history(start='2018-03-01', end='2021-02-25' , interval = '1d')

df = dfa

In [ ]:
data = df
position=0
position_index=0
target= 30
stoploss= 1
realised_pnl=0
number_of_trades=0
pnl_l = 0
pnl_h = 0
pnl_l1 = []
position_l = []
position_2 = []
tradebook = {}

In [ ]:
df

In [ ]:
def BollingerBands(df1, pv=0):
    df1['upper_band'], df1['middle_band'], df1['lower_band'] = ta.BBANDS(df1['Close'], timeperiod =20)
    if pv==1:
        plt.plot(df1['Close'], color = 'black')
        plt.plot(df1['upper_band'], color = 'green')
        plt.plot(df1['lower_band'] , color = 'red')
        plt.show()

In [ ]:
def RSI(df1, pv=0):
    df1['RSI'] = ta.RSI(df1['Close'], timeperiod=14)
    if pv==1:
        plt.plot(df1['RSI'], color = 'blue')
        plt.show

In [ ]:
def MACD(df1, pv=0):
    df1['macd'], df1['macdsignal'], df1['macdhist'] = ta.MACD(df1['Close'], fastperiod=12, slowperiod=26, signalperiod=14)
    if pv==1:
        plt.plot(df1['macd'], color = 'purple')
        plt.plot(df1['macdsignal'], color = 'pink')
        plt.show

In [ ]:
BollingerBands(data)
RSI(data)
MACD(data)


In [ ]:
for i in range(1,(len(data)-1)):

    if ((position == 0)):
         if(((data['Close'][data.index[i]] < data['lower_band'][data.index[i]]) and 
            (data['Close'][data.index[i-1]] > data['lower_band'][data.index[i-1]]))):
                position_time=data.index[i]
                position_index=data.index[i]
                entry_price = data.loc[data.index[i+1],'Open']
                position = 1
                print('long position at {}'.format(data.index[i]))
                number_of_trades=number_of_trades+1
                position_l.append(position_index)
                continue

    if (position_index!=0):
         pnl = ((data.loc[data.index[i],'Close']- entry_price)/entry_price)*(position)*100  
#        

    if ((position != 0)):
        if(#(pnl > target)
           ((data['macd'][data.index[i]] < data['macdsignal'][data.index[i]]) and
           (data['macd'][data.index[i-1]] > data['macdsignal'][data.index[i-1]])) 
          
           or
           (pnl < -stoploss)
        ):
            realised_pnl=realised_pnl+pnl
            pnl_l1.append(pnl)
            position=0
            print('position exited at {}'.format(data.index[i]))
            position_index=0
            tradebook[number_of_trades] = {'entry_time' :position_time ,'entry_price': entry_price ,'long_short': position ,'exit_time' :data.index[i], 'pnl' :pnl}
            print(pnl)
            
        

print(realised_pnl)

In [ ]:
tradebook_df = pd.DataFrame(tradebook).T
tradebook_df['Profit'] = (tradebook_df['entry_price']*tradebook_df['pnl'])/100

In [ ]:
tradebook_df

In [ ]:
realised_pnl

In [ ]:
plt.plot()
plt.plot(np.cumsum(pnl_l1))

In [ ]:
#Using the strategy in the jupyter notebook
ticker = 'TATAMOTORS'
print(ticker)
ticker = ticker + '.NS'
a1 = yf.Ticker(ticker)
df = a1.history(period="60d" , interval = '5m')

high_1 = 1000
low_1 = -1000
data = df
position=0
position_index=0
target= 30
stoploss= 1
realised_pnl=0
number_of_trades=0
pnl_l = 0
pnl_h = 0
pnl_l1 = []
position_l = []
tradebook = {}

for i in range(len(data)):
    if (data.index[i].time()  == dt.time(10,0)):
        high_1 = df.Close.rolling(9).max()[df.index[i]]
        low_1 = df.Close.rolling(9).min()[df.index[i]] 

    if ((position == 0)&( data.Close.loc[data.index[i]]> high_1 ) & (data.index[i].time() > dt.time(10,0))& (data.index[i].time() < dt.time(12,0))):
            position_time=data.index[i].time()
            position_index=data.index[i]
            entry_price = data.loc[data.index[i+1],'Open']
            position = 1
            print('long position at {}'.format(data.index[i]))
            number_of_trades=number_of_trades+1
            position_l.append(position_index)
            continue
    if ((position == 0)&( data.Close.loc[data.index[i]]< low_1 ) & (data.index[i].time() > dt.time(10,0))& (data.index[i].time() < dt.time(12,0))):
            position_time=data.index[i].time()
            position_index=data.index[i]
            entry_price = data.loc[data.index[i+1],'Open']
            position = -1
            print('short position at {}'.format(data.index[i]))
            number_of_trades=number_of_trades+1
            position_l.append(position_index)
            continue


    if (position_index!=0):

        pnl = ((data.loc[data.index[i],'Close']- entry_price)/entry_price)*(position)*100  
#         print('PNL is {}'.format(pnl))


    if ((position != 0) and ((pnl  >target) or (pnl < -stoploss))):
        realised_pnl=realised_pnl+pnl
        pnl_l1.append(pnl)
        position=0
        print('position exited at {}'.format(data.index[i].time()))
        position_index=0
        tradebook[number_of_trades] = {'entry_time' :position_time ,'entry_price': entry_price ,'long_short': position ,'exit_time' :data.index[i], 'pnl' :pnl}
        print(pnl)
    if ((position != 0) and (data.index[i].time() > dt.time(15,0))):
        realised_pnl=realised_pnl+pnl
        pnl_l1.append(pnl)
        tradebook[number_of_trades] = {'entry_time' :position_time ,'entry_price': entry_price ,'long_short': position ,'exit_time' :data.index[i], 'pnl' :pnl}
        position=0
        print('position covered at {}'.format(data.index[i].time()))
        position_index=0
        print(pnl)
        pnl = 0
print(realised_pnl)

In [ ]:
tradebook_df = pd.DataFrame(tradebook).T
tradebook_df['Profit'] = (tradebook_df['entry_price']*tradebook_df['pnl'])/100

In [ ]:
tradebook_df

In [ ]:
realised_pnl

In [ ]:
plt.plot()
plt.plot(np.cumsum(pnl_l1))